# 数据分析

拟定方案：用模板匹配构建 mention => relation 字典

注：方案未完全实现

### 初始化

In [190]:
include("../src/CCKS-mKGQA.jl")
include("../src/tools/regexpr.jl")
include("../src/process/translate.jl")
include("../src/tools/xlsx.jl")
include("../src/loaddata/extractdata.jl") # 原始数据
include("../src/loaddata/translatedata.jl") # 翻译数据
include("../src/tools/ettalign.jl") # 对齐工具
include("../src/datatype.jl") # 数据结构
include("../src/tools/distance.jl")

chardist

In [191]:
## include ILLs files
illpath = "ILLs_4"
include("../src/loaddata/illsdata.jl");

In [109]:
edges = DefaultDict{String, Vector{Tuple}}(Vector{Tuple})
for (_, sub, rel, obj) in raw_triples
    push!(edges[sub], (rel, obj))
end

In [27]:
# 预处理
# mt_train_ques = first.(unique!(mt_train_ques_rels))
mt_train_ques = replace.(lowercase.(mt_train_ques), '?' =>"", "do you know " => "", r"(.*)/ .*" => s"\g<1>");

### 精确匹配

In [145]:
# 训练集已有关系
dict_precise_ques = DefaultDict{String, Vector{Tuple}}(Vector{Tuple})
for (que, _, rels...) in mt_train_ques_rels
    Tuple(rels) ∈ dict_precise_ques[que] && continue
    push!(dict_precise_ques[que], Tuple(rels))
end

In [ ]:
que = "where does the man who influenced NER work?"
dict_precise_ques[que]

2-element Vector{Tuple}:
 ("influences", "work_institutions")
 ("influences", "workplaces")

### 去停用词

In [3]:
# 去停用词
function simplify_que(que)
    que = strip(replace(que, rules...))
    newque = strip(replace(que, rules...))
    while newque != que
        que = newque
        newque = strip(replace(que, rules...))
    end
    que
end

function quesinfo(id)
    println("原始问句 | 翻译问句 | 简化问句 | 提交三元组")
    println(train_ques_ner[id][1])
    println(mt_train_ques[id])
    println(current_ques[id])
    println.(train_sols[id])
end

quesinfo (generic function with 1 method)

In [4]:
middlewords = "(does|did|does|do|are|is|was|he|she|her|his|they|the|that|a|an|of" # 介词助动词
middlewords *= "|[\\d-]+|used to|belong(s|)( to|)|involved (in|)|zoning" # 短语
middlewords *= "|in|to|ner's|at|by|ners|ner|one|from)"

rules = (
    r"(.*)/ .*" => s"\g<1>", # 重复问句
    r"(who did a job|do you know|the car )" => "", # 平凡短语
    r", (a|an) .*?[,?]" => "", # 修饰词
    "'s " => " ",
    r"'s$" => "",
    Regex("^$middlewords ") => "",
    Regex(" $middlewords ") => " ",
    Regex(" $middlewords\$") => "",
    "united states" => "",
    r"(which|where|who|when|what's|what|how|whose) " => " ", # 疑问词
    "?" => "",
    "," => "",
    "-" => " ",
    r"\(.*\)" => "",
    r" {2,}" => " ", # 消除空格
);

In [5]:
# 处理训练集
# 问题集
current_ques = simplify_que.(mt_train_ques)
# 问题关系
current_rels = [querel[3:end] for querel in mt_train_ques_rels]
traversed = Set{String}()
jump2, jump3 = Tuple{Int, String, Vector{String}}[], Tuple{Int, String, Vector{String}}[]
for (i, (que, rels)) in enumerate(zip(current_ques, current_rels))
    txt = join([que,'|', rels...], '\t')
    # txt ∈ traversed ? continue : push!(traversed, txt)
    length(rels) == 2 ? push!(jump2, (i, que, rels)) : push!(jump3, (i, que, rels))
end
sort!(jump2; by=i->length(split(i[2], ' ')))
sort!(jump3; by=i->length(split(i[2], ' ')));

### 方案4-辅助生成

In [55]:
# 筛选两单词问题
# ind men rels
test_ques = filter(i->length(split(i[2]))==2, jump2)

# 从两单词问题提取正则模板
tpls = Dict{Int, Regex}()

# 训练集的 mention => relation 字典
train_men2rel = Dict{Int, Dict}()
# traversed_inds = fill(false, length(mt_train_ques))
traversed = Set{Dict}()

Set{Dict}()

In [ ]:
# 生成模板
testind = 1241
# 提取 mentions
ind, mens, (rel1, rel2) = test_ques[testind]
men1, men2 = String.(split(mens))

# 若反向匹配更优
if chardist(men1, rel1) + chardist(men2, rel2) ≥ chardist(men1, rel2) + chardist(men2, rel1)
    men1, men2 = men2, men1
end

# 建立字典
train_men2rel[ind] = men2rel = Dict(men1 => rel1, men2 => rel2)

# 生成模板
que = mt_train_ques[ind]
if men1 == men2
    println(ind, '\t', que)
else
    tpls[testind] = tpl = Regex('^' * replace(que, men2 => "(?P<men2>.*)", men1 => "(?P<men1>.*)", r"\(.*\)" => "") * '$')
    println("question:\t", que)
    println("mention => rels:\t", men2rel)
    println("template:\t", tpl)
end

question:	where is the capital (or provincial capital) of the country (city) affected by ner
mention => rels:	Dict("affected" => "countries_affected", "capital" => "capital")
template:	r"^where is the (?P<men2>.*)  (?P<men1>.*) by ner$"


In [31]:
# 将模板应用于训练集
inds = findall(ismatch(tpl), mt_train_ques)
newdicts = Set{Dict}()
for ind in inds
    # 提取关系
    que = mt_train_ques[ind]
    length(que) == 5 && continue # 三跳问题略过
    dict = match(tpl, que)
    men1, men2 = String.([dict["men1"], dict["men2"]])
    rel1, rel2 = mt_train_rels[ind]
    max(length(split(men1)), length(split(men2))) > 3 && continue # 字符过长略过
    # 记录关系
    train_men2rel[ind] = men2rel = Dict(men1 => rel1, men2 => rel2)
    men2rel ∈ values(newdicts) || push!(newdicts, men2rel)
end
union!(traversed, newdicts)
println(length(newdicts))

In [67]:
# 筛选两单词问题
# ind men rels
test_ques = filter(i->length(split(i[2]))==2, jump2)

# 从两单词问题提取正则模板
tpls = Dict{Int, Regex}()
invalids = ["does", "did", "does", "do", "are", "is", "was", "he", "she", "her", "his", "they", "the", "that", "a", "an", "of", "in", "to", "ner's", "at", "by", "ners", "ner", "one", "from"];

In [9]:
pairs = sort!(unique!(vcat(collect.(traversed)...)), by=i->length(first(i)));

In [10]:
dict_io = open("analyse/dict_men2rel.txt", "w")
for (key, val) in filter(i->i[1]==i[2], pairs)
    println(dict_io, key, '\t', val)
end
for (key, val) in filter(i->i[1]!=i[2], pairs)
    println(dict_io, key, '\t', val)
end
close(dict_io)

que_io = open("analyse/que_men2rel.txt", "w")
for vals in sort!(collect.(traversed); by = i -> min(length(i[1][1]), length(i[2][1])))
    println(que_io, vals)
end
close(que_io)